In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup   
from urllib.parse import urlparse, parse_qs
import re
import os

In [2]:
def get_df(url,search_word):
    prod_title=[]
    prices=[]
    prod_ratings=[]
    description=[]
    num_ratings=[]
    num_reviews=[]
    prod_id=[]
    images=[]
    seller=[]
    seller_ratings=[]

    req=requests.get(url)
    print(req)
    content=BeautifulSoup(req.content,"html.parser")
    box=content.find("div",class_='_1YokD2 _2GoDe3')
    
    data=box.find_all('div',{'class':'_4ddWXP'})
    links=[]
    start_link="https://www.flipkart.com"
    
    #TITLE
    for items in data:
        rest_link=items.find('a')['href']
        name=items.find('a',attrs={'class':'s1Q9rs'})
        prod_title.append(name.text)
        links.append(start_link+rest_link)
        
    #PRODUCT RATINGS
    for items in data:
        rating_div = items.find('div', class_='_3LWZlK')
        if rating_div:
            prod_ratings.append(rating_div.text.strip())
        else:
            prod_ratings.append('NA')
    
    #PRICES
    def get_prices(soup):
        pri=soup.find_all('div',class_='_30jeq3 _16Jk6d')
        if pri:
            for i in pri:
                pri_only=i.text
                prices.append(pri_only)
        else:
            prices.append("NA")
        return prices
    
    #PRODUCT ID
    def get_prodid(i):
        parsed_url = urlparse(i)
        query_params = parse_qs(parsed_url.query)
        pid_param = query_params.get('pid', None)
        if pid_param:
            pid_value = pid_param[0]
            prod_id.append(str(pid_value))
        else:
            prod_id.append('NA')
        return prod_id
            
    #IMAGES
    def get_images(soup):
        uls = soup.find_all('ul',class_='_3GnUWp')
        if uls:
            # Iterate over each unordered list
            for ul in uls:
                image_list=[]

                # Find all img tags within the unordered list
                imgs = ul.find_all('img')

                # Extract the source (src) attribute of each image and print it
                for img in imgs:
                    image_list.append(img['src'])
                images.append(image_list)

        else:
            imgs=soup.find_all('img',class_='_396cs4 _2amPTt _3qGmMb')
            image_list2=[]

            for img in imgs:
                image_list2.append(img['src'])
            images.append(image_list2)
        return images
    
    #DESCRIPTION
    def get_desc(soup):
        desc=soup.find_all('div',class_='_2cM9lP')
        if desc:
            for i in desc:
                desc_only=i.text
                description.append(desc_only)
        else:
            description.append("NA")
        return description
    
    #SELLER
    def get_seller(soup):
        sel=soup.find('div',class_='_1RLviY')
        if sel:
            second_span = sel.find('span').find_next('span') 
            seller.append(second_span.text.strip())
        else:
            seller.append("NA") 
        return seller
    
    #SELLER RAINGS
    def get_seller_ratings(soup):
        selrat=soup.find_all('div',class_='_3LWZlK _1D-8OL')
        if selrat:
            for i in selrat:
                selrat_only=i.text
                seller_ratings.append(selrat_only)
        else:
            seller_ratings.append('No rating')
        return seller_ratings
    
    #REVIEWS & RATINGS
    def get_reveiws_ratings(soup):
        contents= soup.find_all('div', {"class": "row _2afbiS"})
        if contents:
            for content in contents:
                span = content.find('span')
                if span and 'Reviews' in span.text:
                    reviews_text = span.text
                    num_rev.append(reviews_text)
                if span and 'Ratings' in span.text:
                    ratings_text = span.text
                    num_rat.append(ratings_text[:-2])
        else:
            num_rev.append('0 Reviews')
            num_rat.append('0 Ratings')
        return num_rev,num_rat
    

    
    num_rev=[]
    num_rat=[]
    
    for i in links:
        r=requests.get(i)
        soup=BeautifulSoup(r.text,"html.parser")
        
        fn_prices=get_prices(soup)
        fn_prodid=get_prodid(i)
        fn_images=get_images(soup)
        fn_desc=get_desc(soup)
        fn_seller=get_seller(soup)
        fn_seller_ratings=get_seller_ratings(soup)
        fn_num_rev,fn_num_rat=get_reveiws_ratings(soup)

    # Check lengths of lists
    print("Length of fn_prod_id:", len(fn_prodid))
    print("Length of fn_prices:", len(fn_prices))
    print("Length of ratings:", len(prod_ratings))
    print("Length of fn_images:", len(fn_images))
    print("Length of fn_description:", len(fn_desc))
    print("Length of fn_seller:", len(fn_seller))
    print("Length of fn_seller_ratings:", len(fn_seller_ratings))
    print("Length of fn_num_reviews:", len(fn_num_rev))
    print("Length of fn_num_ratings:", len(fn_num_rat))
    print("\n")
    
    df=pd.DataFrame({"Search Word":search_word,"Product Title":prod_title,"Prices":fn_prices,"Rating":prod_ratings,"Description":fn_desc,"Num of Ratings":fn_num_rat,"Num of Reviews":fn_num_rev,"Product Id":fn_prodid,"Images":fn_images,"Seller":fn_seller,"Seller Ratings":fn_seller_ratings})
    df

    return df


In [3]:
base_url="https://www.flipkart.com/search?q="

search_word=['novels','supplements',"mattresses","gaming-consoles","cameras"]

url=[]
for i in search_word:
    url.append(base_url+i)
print(url)



['https://www.flipkart.com/search?q=novels', 'https://www.flipkart.com/search?q=supplements', 'https://www.flipkart.com/search?q=mattresses', 'https://www.flipkart.com/search?q=gaming-consoles', 'https://www.flipkart.com/search?q=cameras']


In [4]:
df_list=[]
for i,j in zip(url,search_word):
    df_list.append(get_df(i,j))

<Response [200]>
Length of fn_prod_id: 40
Length of fn_prices: 40
Length of ratings: 40
Length of fn_images: 40
Length of fn_description: 40
Length of fn_seller: 40
Length of fn_seller_ratings: 40
Length of fn_num_reviews: 40
Length of fn_num_ratings: 40


<Response [200]>
Length of fn_prod_id: 40
Length of fn_prices: 40
Length of ratings: 40
Length of fn_images: 40
Length of fn_description: 40
Length of fn_seller: 40
Length of fn_seller_ratings: 40
Length of fn_num_reviews: 40
Length of fn_num_ratings: 40


<Response [200]>
Length of fn_prod_id: 40
Length of fn_prices: 40
Length of ratings: 40
Length of fn_images: 40
Length of fn_description: 40
Length of fn_seller: 40
Length of fn_seller_ratings: 40
Length of fn_num_reviews: 40
Length of fn_num_ratings: 40


<Response [200]>
Length of fn_prod_id: 40
Length of fn_prices: 40
Length of ratings: 40
Length of fn_images: 40
Length of fn_description: 40
Length of fn_seller: 40
Length of fn_seller_ratings: 40
Length of fn_num_reviews: 40
Len

In [14]:
# Filter DataFrames in the list based on the length of the "images" list
filtered_df_list = [df[df['Images'].apply(len) >= 3] for df in df_list]

# Reset the index of each filtered DataFrame
filtered_df_list = [df.reset_index(drop=True) for df in filtered_dataframes]


In [15]:
result_df=pd.concat(filtered_dataframes, ignore_index=True)
result_df

,Search Word,Product Title,Prices,Rating,Description,Num of Ratings,Num of Reviews,Product Id,Images,Seller,Seller Ratings
0,novels,Complete Series Of Pery Jackson(Set Of 5 Books)C,₹849,4.2,HighlightsAuthor: Percy Jackson1685 PagesLangu...,169 Ratings,16 Reviews,RBKG66KPSYYEEZG3,[https://rukminim2.flixcart.com/image/128/128/...,CreativeworldStore,3.9
1,novels,The Diary Of A Young Girl | All Time Great Cla...,₹99,4.4,HighlightsAuthor: Sawan168 PagesLanguage: Engl...,61 Ratings,7 Reviews,RBKFY8H9RFNZKMMG,[https://rukminim2.flixcart.com/image/128/128/...,GOWOO,4.6
2,novels,Two Indian Girls - Book 1 of The Kanke Killin...,₹202,4.1,HighlightsLanguage: EnglishBinding: PaperbackP...,208 Ratings,15 Reviews,9798758994283,[https://rukminim2.flixcart.com/image/128/128/...,AuthorKinshuk,4.5
3,novels,Ritualistic Murder - A Prequel (Book 0) to Th...,₹214,4.2,HighlightsLanguage: EnglishBinding: PaperbackP...,118 Ratings,20 Reviews,9798753694072,[https://rukminim2.flixcart.com/image/128/128/...,AuthorKinshuk,4.5
4,novels,Uzumaki (3-in-1 Deluxe Edition),₹380,3.9,HighlightsLanguage: EnglishBinding: PaperbackP...,378 Ratings,42 Reviews,9781421561325,[https://rukminim2.flixcart.com/image/128/128/...,harinovels,No rating
...,...,...,...,...,...,...,...,...,...,...,...
149,cameras,DIGITEK GoCAM DAC-101 5K 30FPS WiFi Ultra HD w...,"₹8,999",3.4,HighlightsEffective Pixels: 48 MP5K,188 Ratings,22 Reviews,SAYGMPTRDYSMEDUG,[https://rukminim2.flixcart.com/image/128/128/...,RetailNet,4.7
150,cameras,Bzrqx MINI CAMERA Spy WiFi Mini Hidden Camera ...,₹799,4.2,HighlightsEffective Pixels: 12 MP1920*1080p,5 Ratings,0 Reviews,SAYGG7Q9QT8ZYKHJ,[https://rukminim2.flixcart.com/image/128/128/...,DealssShop,4.5
151,cameras,GoPro HERO8 Black Sports and Action Camera,"₹26,990",4.2,HighlightsEffective Pixels: 12 MP4K60+1080p240,"1,091 Ratings",140 Reviews,SAYFKYD4UE2GYZJR,[https://rukminim2.flixcart.com/image/128/128/...,RetailNet,4.7
152,cameras,dji Action 2 Dual Screen Combo Dual Screen Com...,"₹20,990",4,HighlightsEffective Pixels: 12 MP4K,70 Ratings,8 Reviews,SAYG8YYVA7GPKJ7Z,[https://rukminim2.flixcart.com/image/128/128/...,RetailNet,4.7


In [18]:
# Iterate over each DataFrame and its corresponding search word
for df, word in zip(filtered_df_list, search_word):
    # Specify the folder path based on the search word
    folder_path = rf'D:\flipkart_webstories_json\{word} json'
    os.makedirs(folder_path, exist_ok=True)

    # Iterate over each row of the DataFrame
    for index, row in df.iterrows():
        # Convert the row to a JSON string
        json_string = row.to_json()

        # Define the file name with ".json" extension
        file_name = os.path.join(folder_path, f"{row['Product Id']}.json")

        # Write the JSON string to a file
        with open(file_name, 'w') as json_file:
            json_file.write(json_string)

    print(f"JSON files for '{word}' created successfully.")

JSON files for 'novels' created successfully.
JSON files for 'supplements' created successfully.
JSON files for 'mattresses' created successfully.
JSON files for 'gaming-consoles' created successfully.
JSON files for 'cameras' created successfully.
